In [1]:
import RetrievalAndSearch.minsearch as minsearch
import json
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [2]:
def read_documents(doc_path):
    with open(doc_path, 'rt') as f_in:
        docs_raw = json.load(f_in)

    documents = []

    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)

    return documents

In [3]:
def search(query, documents):
    es_client = Elasticsearch("http://localhost:9200")
    
    index_name = "course-questions-de-zoomcamp"

    if not es_client.indices.exists(index=index_name):

        index_settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "text": {"type": "text"},
                    "section": {"type": "text"},
                    "question": {"type": "text"},
                    "course": {"type": "keyword"} 
                }
            }
        }
        
        es_client.indices.create(index=index_name, body=index_settings)
    
        for doc in tqdm(documents):
            es_client.index(index=index_name, document=doc)

    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    results = []

    for hit in response['hits']['hits']:
        results.append(hit['_source'])

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [5]:
def llm(prompt):
    client = OpenAI()
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content':prompt}]
    )

    return response.choices[0].message.content

In [6]:
def rag(query, doc_path):
    documents = read_documents(doc_path)
    search_results = search(query, documents)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
doc_path = 'RetrievalAndSearch/documents.json'
query = "Can I still enroll the course?"
rag(query, doc_path)

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homework despite not registering. Be mindful of the deadlines for the final projects.'

In [ ]:
# RUNNING ELASTICSEARCH IN DOCKER

#docker run -it \
#    --rm \
#    --name elasticsearch \
#    -p 9200:9200 \
#    -p 9300:9300 \
#    -e "discovery.type=single-node" \
#    -e "xpack.security.enabled=false" \
#    docker.elastic.co/elasticsearch/elasticsearch:8.4.3